In [1]:
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
from time import sleep

from datetime import datetime, timedelta
from random import randrange

%load_ext dotenv
%dotenv

In [2]:
def _get_elements(driver: webdriver.Chrome, selector: str, return_first: bool = True) -> WebElement | list[WebElement]:
    for _ in range(100):
        elements = driver.find_elements(By.XPATH, selector)
        if elements:
            sleep(0.2)
            return elements[0] if return_first else elements
        else:
            sleep(0.1)
    raise Exception(f'Element by selector {selector} not found.')

def get_element(driver: webdriver.Chrome, selector: str) -> WebElement:
    res = _get_elements(driver, selector, True)
    if isinstance(res, list):
        raise
    return res

def get_elements(driver: webdriver.Chrome, selector: str) -> list[WebElement]:
    res = _get_elements(driver, selector, True)
    if isinstance(res, WebElement):
        raise
    return res


class AFIP:
    def __init__(self, headless:bool = False) -> None:
        self.headless = headless
        self.home_container = None
        
        self.home_container = None

        current_dir = os.getcwd()
        target_url = 'https://afip.gob.ar/'
        with open('blank.html', 'w') as f:
            f.write(f'<a href="{target_url}" target="_blank">link</a>')

        option = webdriver.ChromeOptions()
        option.add_argument("start-maximized")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
        self.driver.get(f'file://{current_dir}/blank.html')
        links = self.driver.find_elements(By.XPATH, "//a[@href]")
        sleep(0.5)
        links[0].click()
        sleep(0.5)
        self.driver.switch_to.window(self.driver.window_handles[-1])

        sleep(2)
        self.login()

        sleep(1)
        self.go_to_linea()

        sleep(0.5)

        if not self.driver:
            raise Exception('Driver not initialized')
    
    def login(self):
        login_btn = get_element(self.driver, '//a[@href="https://auth.afip.gob.ar/contribuyente_/login.xhtml"]')
        login_btn.click()
        self.driver.switch_to.window(self.driver.window_handles[-1])

        login_field = get_element(self.driver, '//input[@id="F1:username"]')
        login_field.send_keys(os.getenv('CUIT'))

        submit_btn = get_element(self.driver, '//input[@id="F1:btnSiguiente"]')
        submit_btn.click()

        pass_field = get_element(self.driver, '//input[@id="F1:password"]')
        pass_field.send_keys(os.getenv('PASS'))

        submit_btn = get_element(self.driver, '//input[@id="F1:btnIngresar"]')
        submit_btn.click()
    
    def go_to_linea(self):

        linea_btn = get_element(self.driver, '//a[.//div[.//h3[text()="Comprobantes en línea"]]]')
        linea_btn.click()
        sleep(1)
        self.driver.switch_to.window(self.driver.window_handles[-1])
        sergei_btn = get_element(self.driver, '//input[@value="NAZAROV SERGEI"]')
        sergei_btn.click()

    def make_invoice(self, date: str, price: int | str):
        def click_continuar():
            continuar_btn = get_element(self.driver, '//input[@value="Continuar >"]')
            continuar_btn.click()

        generar_btn = get_element(self.driver, '//a[@id="btn_gen_cmp"]')
        generar_btn.click()

        punto_de_ventas = get_element(self.driver, '//select[@name="puntoDeVenta"]')
        select = Select(punto_de_ventas)
        select.select_by_index(1)

        click_continuar()

        # filling dates page
        date1 = get_element(self.driver, '//input[@id="fc"]')
        date1.clear()
        date1.send_keys(date)

        conceptos = get_element(self.driver, '//select[@id="idconcepto"]')
        select = Select(conceptos)
        select.select_by_index(2)

        date2 = get_element(self.driver, '//input[@id="fsd"]')
        date2.clear()
        date2.send_keys(date)

        date3 = get_element(self.driver, '//input[@id="fsh"]')
        date3.clear()
        date3.send_keys(date)

        date4 = get_element(self.driver, '//input[@id="vencimientopago"]')
        date4.clear()
        date4.send_keys(date)

        click_continuar()

        # filling serviosios page

        # Consumidor final
        cons_fin = get_element(self.driver, '//select[@id="idivareceptor"]')
        select = Select(cons_fin)
        select.select_by_index(3)

        # Otra checkbox
        otra_chb = get_element(self.driver, '//input[@id="formadepago7"]')
        otra_chb.click()

        click_continuar()

        # filling table page
        producto_text = get_element(self.driver, '//textarea[@id="detalle_descripcion1"]')
        producto_text.clear()
        producto_text.send_keys('Servicios prestados')

        # filling unidades
        unidades = get_element(self.driver, '//select[@id="detalle_medida1"]')
        select = Select(unidades)
        select.select_by_index(7)

        # filling precio
        precio = get_element(self.driver, '//input[@id="detalle_precio1"]')
        precio.clear()
        precio.send_keys(str(price))

        click_continuar()

        # final page
        confirmar_datos = get_element(self.driver, '//input[@value="Confirmar Datos..."]')
        confirmar_datos.click()

        # ожидание появления окна подтверждения
        wait = WebDriverWait(self.driver, 10)
        wait.until(EC.alert_is_present())

        # переключаемся на окно подтверждения и нажимаем "OK"
        alert = self.driver.switch_to.alert
        alert.accept()
        self.driver.switch_to.default_content()

        menu_principal = get_element(self.driver, '//input[@value="Menú Principal"]')
        menu_principal.click()
        sleep(0.5)


    def close(self) -> None:
        if self.driver is not None:
            self.driver.quit()
    

In [3]:
import json

def save_data_to_file(data, file_path):
    """
    Save the data object to a file in JSON format.

    :param data: The data object to be saved.
    :param file_path: The path of the file where the data will be saved.
    """
    with open(file_path, 'w') as file:
        json.dump(data, file)

def read_data_from_file(file_path):
    """
    Read data from a JSON file and convert it into a Python object.

    :param file_path: The path of the file from which to read the data.
    :return: The Python object obtained from the JSON file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)


In [4]:
# Initial conditions
start_date = datetime(2024, 4, 1)  # Start
total_days = 30  # Number of days in December
holidays = {1, 2, 3, 4, 5, 6, 7}  # Holidays
total_amount = randrange(970, 995)  # Total amount in thousands
maximum_day_amount = 90000
minimum_day_amount = 50000
data = []  # Initial list
daily_amounts = []  # List to store amounts for each working day

# Check when December starts (on Friday in 2023)
weekday = start_date.weekday()

# Generate amounts for working days, accounting for holidays
for day in range(1, total_days + 1):
    current_date = start_date + timedelta(days=day - 1)
    # Check if the day is a working day (Mon-Fri) and not a holiday
    if current_date.weekday() < 5 and current_date.day not in holidays:
        # Generate amounts that are multiples of thousands and do not exceed 39000
        daily_amounts.append(randrange(25, 40) * 1000)

# Algorithm to distribute amounts to keep the total amount within the specified bounds
# This may require multiple attempts and adjustments based on the number of working days
while sum(daily_amounts) != total_amount * 1000:
    for i in range(len(daily_amounts)):
        # Make adjustments considering the maximum amount
        if sum(daily_amounts) < total_amount * 1000 and daily_amounts[i] < maximum_day_amount:
            daily_amounts[i] += 1000
        elif sum(daily_amounts) > total_amount * 1000 and daily_amounts[i] > minimum_day_amount:
            daily_amounts[i] -= 1000

# Populate the final list with data
current_day = 0
for day in range(1, total_days + 1):
    current_date = start_date + timedelta(days=day - 1)
    if current_date.weekday() < 5 and current_date.day not in holidays:
        data.append(dict(date=current_date.strftime('%d/%m/%Y'), price=str(daily_amounts[current_day])))
        current_day += 1

# Display the list
summ = 0
for i, entry  in enumerate(data):
    summ += int(entry['price'])
    print(f'{i}  {entry}')
print(summ)
save_data_to_file(data, 'data.json')

0  {'date': '08/04/2024', 'price': '55000'}
1  {'date': '09/04/2024', 'price': '54000'}
2  {'date': '10/04/2024', 'price': '56000'}
3  {'date': '11/04/2024', 'price': '57000'}
4  {'date': '12/04/2024', 'price': '61000'}
5  {'date': '15/04/2024', 'price': '57000'}
6  {'date': '16/04/2024', 'price': '64000'}
7  {'date': '17/04/2024', 'price': '62000'}
8  {'date': '18/04/2024', 'price': '62000'}
9  {'date': '19/04/2024', 'price': '56000'}
10  {'date': '22/04/2024', 'price': '54000'}
11  {'date': '23/04/2024', 'price': '65000'}
12  {'date': '24/04/2024', 'price': '56000'}
13  {'date': '25/04/2024', 'price': '51000'}
14  {'date': '26/04/2024', 'price': '62000'}
15  {'date': '29/04/2024', 'price': '59000'}
16  {'date': '30/04/2024', 'price': '55000'}
986000


In [7]:
data = read_data_from_file('data.json')
summ = 0
for i, entry  in enumerate(data):
    summ += int(entry['price'])
    print(f'{i}  {entry}')
print(summ)

0  {'date': '08/04/2024', 'price': '55000'}
1  {'date': '09/04/2024', 'price': '54000'}
2  {'date': '10/04/2024', 'price': '56000'}
3  {'date': '11/04/2024', 'price': '57000'}
4  {'date': '12/04/2024', 'price': '61000'}
5  {'date': '15/04/2024', 'price': '57000'}
6  {'date': '16/04/2024', 'price': '64000'}
7  {'date': '17/04/2024', 'price': '62000'}
8  {'date': '18/04/2024', 'price': '62000'}
9  {'date': '19/04/2024', 'price': '56000'}
10  {'date': '22/04/2024', 'price': '54000'}
11  {'date': '23/04/2024', 'price': '65000'}
12  {'date': '24/04/2024', 'price': '56000'}
13  {'date': '25/04/2024', 'price': '51000'}
14  {'date': '26/04/2024', 'price': '62000'}
15  {'date': '29/04/2024', 'price': '59000'}
16  {'date': '30/04/2024', 'price': '55000'}
986000


In [8]:
summ = 0
data = data[:2]
for i, entry  in enumerate(data):
    summ += int(entry['price'])
    print(f'{i}  {entry}')
print(summ)

0  {'date': '08/04/2024', 'price': '55000'}
1  {'date': '09/04/2024', 'price': '54000'}
109000


In [9]:
################ DANGER!!!!!!!!!!!!!!!!!!!!!!!
wp = AFIP()
for el in data:
    print(f"making invoice on {el['date']} ({el['price']} pesos)...")
    wp.make_invoice(date=el['date'], price=el['price'])
print('done!')
wp.close()

making invoice on 20/03/2024 (52000 pesos)...
making invoice on 21/03/2024 (50000 pesos)...
making invoice on 22/03/2024 (45000 pesos)...
making invoice on 25/03/2024 (46000 pesos)...
making invoice on 26/03/2024 (47000 pesos)...
done!
